# PDF renamer with Llama 2 (Python)

This Jupyter Notebook is meant to be used in Google Colab, but can be easily run locally.  
The aim of this Jupyter Notebook is to rename PDFs based on the title contained inside its own text. Especially regarding scientific publications.

Based on a previous notebook:

## Setting up Llama 2

In [1]:
!pip install llama-cpp-python==0.1.78

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 9.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 5.0 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.1.78-cp310-cp310-linux_x86_64.whl size=296587 sha256=9f4925a8af27fb87c1aff3da0c77eef282bed4a188051074c222c80281411267
  Stored in directory: /root/.cache/pip/wheels/61/f9/20/9ca660a9d3f2a47e44217059409478865948b5c8a1cba70030
Successfully built llama-cpp-python


If something goes wrong here or later, please check the most up-to-date version of llama-cpp-python at the following link:

https://pypi.org/project/llama-cpp-python/

In [2]:
from llama_cpp import Llama

from IPython.display import display, HTML
import json
import time
import pathlib

In [3]:
!pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=9f29170c0ab8614e04a23d853deea831afa1b16bccf6b28b85a24532f2991f6c
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [4]:
import wget

url = 'https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGML/resolve/main/llama-2-7b-chat.ggmlv3.q8_0.bin'
local_path = '/content/'

wget.download(url, local_path)

'/content//llama-2-7b-chat.ggmlv3.q8_0.bin'

In [5]:
MODEL_Q8_0 = Llama(
    model_path="/content/llama-2-7b-chat.ggmlv3.q8_0.bin",
    n_ctx=2048)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | VSX = 0 | 


## Creation of functions related to reading PDFs

In [6]:
import re

def extract_text_between_quotes(input_string):
    pattern1 = r'"(.*?)"'
    pattern2 = r"'(.*?)'"
    pattern3 = r'(.*?)\sB:'

    match = re.search(pattern1, input_string)
    if match:
        return match.group(1)
    else:
        match = re.search(pattern2, input_string)
        if match:
            return match.group(1)
        else:
            match = re.search(pattern3, input_string)
            if match:
                return match.group(1)
            else:
                return None

In [7]:
def query(model, question):
    model_name = pathlib.Path(model.model_path).name
    time_start = time.time()
    prompt = f"Q: {question} A:"
    output = model(prompt=prompt, max_tokens=0) # if max tokens is zero, depends on n_ctx
    response = output["choices"][0]["text"]
    time_elapsed = time.time() - time_start
    #display(HTML(f'<code>{model_name} response time: {time_elapsed:.02f} sec</code>'))
    #display(HTML(f'<strong>Question:</strong> {question}'))
    display(HTML(f'<strong>Answer:</strong> {response}'))
    #print(json.dumps(output, indent=2))

    response = extract_text_between_quotes(response)
    return response

In [8]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.9 MB/s eta 0:00:00


In [9]:
import PyPDF2
from PyPDF2 import PdfReader

#text extraction
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, "rb") as file:
        pdf_reader = PdfReader(file)
        full_text = ""
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            full_text += page.extract_text()
    return full_text

#title extraction from metadata
def get_pdf_title(pdf_path):
    title = None
    with open(pdf_path, "rb") as file:
        pdf_reader = PyPDF2.PdfReader(file)
        if pdf_reader.metadata is not None:
            title = pdf_reader.metadata.title
    return title

## Testing with manually-labelled samples

In [10]:
import os

def get_pdf_files(directory):
    pdf_files = []
    for file in os.listdir(directory):
        if file.endswith('.pdf'):
            pdf_files.append(os.path.join(directory, file))
    return pdf_files

def process_pdfs_in_directory(directory):
    pdf_files = get_pdf_files(directory)
    for pdf_file in pdf_files:
        text = extract_text_from_pdf(pdf_file)
        text = text[:300]
        title = query(MODEL_Q8_0, "Based on the initial 300 characters of a research paper, determine the title of the paper. Follow the scheme: The title is '...'. Here's the text snippet: '" + text + "'")
        truetitle = get_pdf_title(pdf_file)
        print(f"PDF: {pdf_file} \nTitle: {title}\nTrue Title: {truetitle}\n")
        print('---------------------------------------------------')

In [11]:
# Manually-labelled PDFs ZIP download
main_path = '/content/'

url1 = "https://github.com/diegopastorbonet/PDF-Renamer-with-Llama2/blob/main/PDFs_Test_1.zip?raw=true"
url2 = "https://github.com/diegopastorbonet/PDF-Renamer-with-Llama2/blob/main/PDFs_Test_2.zip?raw=true"
url3 = "https://github.com/diegopastorbonet/PDF-Renamer-with-Llama2/blob/main/PDFs_Test_3.zip?raw=true"
url4 = "https://github.com/diegopastorbonet/PDF-Renamer-with-Llama2/blob/main/PDFs_Test_4.zip?raw=true"
url5 = "https://github.com/diegopastorbonet/PDF-Renamer-with-Llama2/blob/main/PDFs_Test_5.zip?raw=true"

wget.download(url1, main_path)
wget.download(url2, main_path)
wget.download(url3, main_path)
wget.download(url4, main_path)
wget.download(url5, main_path)

'/content//PDFs_Test_5.zip'

In [12]:
import zipfile
import os

# Creation of test PDFs directory
output_path = '/content/PDFs/'
if not os.path.exists(output_path):
    os.makedirs(output_path)

# ZIP files directory
zip_directory = main_path

# Listing every file in the directory
zip_files = [f for f in os.listdir(zip_directory) if f.endswith('.zip')]

# Iteration over ZIPs and PDF extraction
for zip_file in zip_files:
    with zipfile.ZipFile(os.path.join(zip_directory, zip_file), 'r') as zip_ref:
        zip_ref.extractall(output_path)
    print("Archivos PDF extraídos de", zip_file)

    # Erasing original ZIP file
    os.remove(os.path.join(zip_directory, zip_file))
    print("Archivo ZIP original eliminado:", zip_file)

print("Archivos PDF extraídos en:", output_path)

Archivos PDF extraídos de PDFs_Test_1.zip
Archivo ZIP original eliminado: PDFs_Test_1.zip
Archivos PDF extraídos de PDFs_Test_4.zip
Archivo ZIP original eliminado: PDFs_Test_4.zip
Archivos PDF extraídos de PDFs_Test_3.zip
Archivo ZIP original eliminado: PDFs_Test_3.zip
Archivos PDF extraídos de PDFs_Test_5.zip
Archivo ZIP original eliminado: PDFs_Test_5.zip
Archivos PDF extraídos de PDFs_Test_2.zip
Archivo ZIP original eliminado: PDFs_Test_2.zip
Archivos PDF extraídos en: /content/PDFs/


In [13]:
pdf_directory = '/content/PDFs/'

process_pdfs_in_directory(pdf_directory)

PDF: /content/PDFs/iannaccone1984.pdf 
Title: Reviewing the Reviews on Literacy and Reasoning.
True Title: Reviewing the Reviews on Literacy and Reasoning: Some Selected Themes and References

---------------------------------------------------


Llama.generate: prefix-match hit


PDF: /content/PDFs/abadal2020.pdf 
Title: Programmable Metamaterials for Software-Defined Electromagnetic Control
True Title: Guest Editorial: Programmable Metamaterials for Software-Defined Electromagnetic Control

---------------------------------------------------


Llama.generate: prefix-match hit


PDF: /content/PDFs/abdelrahman2020.pdf 
Title: Commentary on 'Voided volume for postoperative voiding assessment following prolapse and urinary incontinence surgery.'
True Title: Commentary on “Voided volume for postoperative voiding assessment following prolapse and urinary incontinence surgery”

---------------------------------------------------


...

PDF: /content/PDFs/goldberg2000.pdf 
Title: A: The impact of Z on W'
True Title: Laser Treatment Of Vascular Lesions

---------------------------------------------------


Llama.generate: prefix-match hit


PDF: /content/PDFs/ahmedjuma2018.pdf 
Title: The Importance of Digital Libraries and Requirements - Case Study: The National Center for Research.
True Title: The Importance of Digital Libraries and Requirements - Case Study: The National Center for Research, Initiative to Establish the Sudanese Digital Library

---------------------------------------------------


Llama.generate: prefix-match hit


PDF: /content/PDFs/gelhausen2020.pdf 
Title: Forecasting Future Air Traffic Development Up to 2040 and Assessing Mitigation Strategies
True Title: Part III. Forecasting future air traffic development up to 2040 and assessing mitigation strategies

---------------------------------------------------


## Use on own PDF

In [ ]:
from google.colab import files

print("Please upload your PDF:")
uploaded_file = files.upload()

In [ ]:
def process_own_pdf(pdf_file):
      text = extract_text_from_pdf(pdf_file)
      text = text[:300]
      title = query(MODEL_Q8_0, "Based on the initial 300 characters of a research paper, determine the title of the paper. Follow the scheme: The title is '...'. Here's the text snippet: '" + text + "'")
      print("Title: " + title)
      print('---------------------------------------------------')

In [ ]:
process_own_pdf("your PDF rute")